## _Monthly Prediction for Asset Returns_

_The following code will implement machine learning models to predict the upcoming returns of the following assets. The idea behind this code will be to run multiple models and get the best model that works for every assets. This method can help on upcoming projects, find the optimal model in the long run for each asset while using tuning of parameters. In this case, tuning is not necessary since there are not enough data points to stretch our models._ 

_Assets are US equities and price return is predicted on their 10-K* quarterly reports. To get even more realistic estimates we future shift this financial results by one quarter, for example, we use Q4 results published in late December to estimate price returns in Q1 of the following year. A big challenge four our project was the following dilemma, transform our returns from monthly to quarterly (upscaling) or transform quarterly earnings in monthly (downscaling). The choice was made easier by our dataset length. We had approximately 10 years of data so 120 points, if we chose the upscaling method our dataset would have had a length of 40 points, basically not enough for machine learning methods (ML). Ergo, we chose the late method, using the same earnings for three month consecutively, meaning that we would have the same results for Jan/Feb/March. We added also some lagged price returns (max 2 lags). Yes, rolling mean was choice too but using rolling mean with a window of two is not that significant._

_Finally we choose a window size of 12 starting points. We train our model on the first 12 months and we predict on the following month. We choose to minimise the MAE, to have a better understanding of our error. We predict the (12 + 1) 13th month, and our step is of 1, meaning that our dataset grows in time. The first prediction uses 12 data points, second one uses 13 data points, and the last one uses the (n - 1) data points (n being the length of the dataset). To be precise the full length of our dataset is n - 2 since we lagged the price by two periods._

_Finally we construct portfolio weights using the returns predicted and back-testing on the real data. This part will be available in the next notebook._


_**A 10-K is a comprehensive report filed annually by public companies about their financial performance.**_ 

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
file_path= "../data/"
# List of equities
list_col = ['HES US Equity', 'NEM US Equity', 'INTC US Equity', 'IFF US Equity',
       'MOS US Equity', 'EXC US Equity', 'KMB US Equity', 'JNJ US Equity',
       'BAX US Equity', 'F US Equity', 'COP US Equity',
       'WFC US Equity', 'WY US Equity', 'TGT US Equity', 'MMM US Equity',
       'CMI US Equity', 'HAS US Equity', 'DUK US Equity',
       'EMN US Equity', 'BK US Equity', 'UFS US Equity', 'ECL US Equity',
       'SLB US Equity', 'UPS US Equity', 'T US Equity', 'NSC US Equity',
       'PPL US Equity', 'MO US Equity', 'JLL US Equity', 'C US Equity',
       'ABT US Equity', 'AMD US Equity', 'CVX US Equity', 'CMA US Equity',
       'DTE US Equity', 'HSY US Equity', 'KIM US Equity', 'NBL US Equity',
       'IBM US Equity', 'WELL US Equity', 'IR US Equity', 'BKR US Equity',
       'WEC US Equity', 'OXY US Equity', 'WMB US Equity', 'UNP US Equity',
       'WM US Equity', 'CCL US Equity']

In [3]:
# Reading all the csv file containing price returns and financial data
returns = pd.read_csv(file_path+"return.csv")[['Date1']+list_col]
returns.rename(columns={returns.columns[0]: "Date" }, inplace = True)
CURRENT_EV_TO_T12M_EBITDA = pd.read_csv(file_path+"CURRENT_EV_TO_T12M_EBITDA.csv")[['Date']+list_col]
DIVIDEND_INDICATED_YIELD= pd.read_csv(file_path+"DIVIDEND_INDICATED_YIELD.csv")[['Date']+list_col]
EBITDA_TO_INTEREST_EXPN = pd.read_csv(file_path+"EBITDA_TO_INTEREST_EXPN.csv")[['Date']+list_col]
EBITDA_TO_REVENUE = pd.read_csv(file_path+"EBITDA_TO_REVENUE.csv")[['Date']+list_col]
EPS_GROWTH = pd.read_csv(file_path+"EPS_GROWTH.csv")[['Date']+list_col]
EQY_DVD_YLD_IND = pd.read_csv(file_path+"EQY_DVD_YLD_IND.csv")[['Date']+list_col]
PE_RATIO = pd.read_csv(file_path+"PE_RATIO.csv")[['Date']+list_col]
TOT_DEBT_TO_EBITDA = pd.read_csv(file_path+"TOT_DEBT_TO_EBITDA.csv")[['Date']+list_col]

In [4]:
# Not taking EPS growth because if NaN values 
list_df_str = [
            'CURRENT_EV_TO_T12M_EBITDA','DIVIDEND_INDICATED_YIELD','EBITDA_TO_INTEREST_EXPN',
            'EBITDA_TO_REVENUE','EQY_DVD_YLD_IND','PE_RATIO','TOT_DEBT_TO_EBITDA'
          ]
list_df = [
            CURRENT_EV_TO_T12M_EBITDA,DIVIDEND_INDICATED_YIELD,EBITDA_TO_INTEREST_EXPN,
            EBITDA_TO_REVENUE,EQY_DVD_YLD_IND,PE_RATIO,TOT_DEBT_TO_EBITDA
          ]
len(list_df)

7

In [5]:
def transform(col, timeline, lags):
    
    """
    This function, will create a dataframe for each asset with the according financial records and the lagged price.
    
    col : name of the column of the asset (string)
    timeline: you can choose between "Monthly" and "Quarterly". We use manually but the option is open.
    lags : Accepting integers for the number of lags to add to the dataset
    
    """
    
    df = returns[["Date",str(col)]].copy()
    df.Date = pd.to_datetime(df.Date)
    df.Date = df.Date.dt.strftime('%m/%Y')
    df_final = df.copy()
    
    for res,res_str in zip(list_df,list_df_str):

        df2 = res[["Date",str(col)]].copy()
        df2.Date = pd.to_datetime(df2.Date)
        df2.rename(columns={str(col): str(res_str) }, inplace = True)
        # Shifting our financial records from Q4 to Q1
        df2.Date = df2.Date + pd.DateOffset(months=1)
        df2.Date = df2.Date.dt.strftime('%m/%Y')
        
        if timeline == 'Monthly':
            # Filling the upcoming months with the previous data
            df_final = pd.merge(df_final,df2,on='Date', how='outer').fillna(method='ffill').dropna()
        elif timeline == 'Quarterly':
            # Merging only on quarterly data.
            df_final = pd.merge(df_final,df2,on='Date', how='outer').dropna()    

    for l in range(1,lags+1):
        # Number of lags
        df_final[col+'lag'+str(l)] = df_final[col].shift(int(l),axis = 0)
        
    df_final.dropna(inplace = True)
    df_final.reset_index(drop=True,inplace = True)
    df_final.Date = pd.to_datetime(df_final.Date)
    
    return df_final

In [6]:
def regressonModels(col, timeline, lags):
    
    """
    This function will take the dataframe created in the previous function and will perform the ML models.
    The inputs are the same as in the previous function since it will be called in this part. 
    
    """
    
    
    # Creation d'une liste avec les modeles qui vont être appliqués
    from xgboost import XGBRegressor
    from sklearn.ensemble import GradientBoostingRegressor
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.ensemble import AdaBoostRegressor
    from catboost import CatBoostRegressor
    from lightgbm import LGBMRegressor
    from sklearn.linear_model import SGDRegressor
    from sklearn import metrics
    from sklearn.preprocessing import StandardScaler

    # Creating a list to rotate models.
    models = []
    models.append(XGBRegressor(silent=True))
    models.append(GradientBoostingRegressor(verbose = 0))
    models.append(RandomForestRegressor(verbose=0))
    models.append(AdaBoostRegressor())
    models.append(CatBoostRegressor(silent=True))
    models.append(LGBMRegressor(silent=True))
    models.append(SGDRegressor())    

    
    df_final = transform(col, timeline, lags)
    y = df_final[[col]].copy()
    date = df_final['Date'].to_list()
    X = df_final.drop(['Date',col],axis=1).copy()
    # Scaling data
    sc = StandardScaler()
    X[X.columns] = sc.fit_transform(X[X.columns])
    
    if timeline == "Monthly":
        window_size = 12
        step_size = 1
        backtest_size = 1
    elif timeline == "Quarterly":
        window_size = 4
        step_size = 1
        backtest_size = 1        
        
        
    i = 0 
    results = []
    date_of_pred = []
    names = []
    
    ##################
    results_y_pred = []
    #################

    for m in models:
        m = str(m).split('(')[0]
        names.append(m)

    while i + window_size <= X.shape[0]:

#         print("Current index"+str(i))

        # We take the first i + window size data points
        sample_X = X.iloc[:i+window_size,:].copy()
        sample_Y = y.iloc[:i+window_size].copy()
        # We train it in the first i + window size - 1 points
        X_train = sample_X.iloc[:-backtest_size,:].copy()
        Y_train = sample_Y.iloc[:-backtest_size].copy()
        # We test for the following month (so we try to predict only one month)
        X_test = sample_X.iloc[-backtest_size:,:].copy()
        Y_test = sample_Y.iloc[-backtest_size:].copy()
        #date_of_pred.append(str(date[X_test.index[-1]]))
        date_of_pred.append(str(date[X_train.index[0]]) + '-' + str(date[X_test.index[-1]]))

        """
        In case we want to calculate more than the next month, this will take the first 
        and the last date of the X_test
        date_of_pred.append(str(date[X_test.index[0]]) + '-' + str(date[X_test.index[-1]]))
        """

        modelscore = []
        y_predit_list = []
        # Rotating models, predicting our t+1, also collecting the MAE for our final report.
        for model in models:

            model.fit(X_train,Y_train)
            y_pred = model.predict(X_test)
            y_predit_list.append(y_pred[0])
            # Mean absolute error
            score = metrics.mean_absolute_error(Y_test, y_pred)
            modelscore.append(score)

        results_y_pred.append(y_predit_list)
        results.append(modelscore)

        i+=step_size
        
    # Returning 
    
    results_summary = pd.DataFrame(results,columns = names)
    #results_summary.insert(0,"period_of_pred",date_of_pred)
    results_summary['Best_model'] = results_summary.idxmin(axis=1)
    best_m = results_summary.head(1)['Best_model'].values[0]
    
    
    results_y_pred = pd.DataFrame(results_y_pred,columns = names)
    val_of_y = list(results_y_pred[best_m])
    
    print('Ticker '+str(col)+' finished')    
    
    return col , best_m , val_of_y

In [1]:
asset_list = []
best_model_list = []
val_of_y_list = []

# This for loop will loop through all the list of dataframes and perform transformation + ML prediction and store those in 
# lists that are created.
for asset in list_col:
    ass , best_m, val_of_y = regressonModels(str(asset), 'Monthly', 2)
    asset_list.append(ass)
    best_model_list.append(best_m)
    val_of_y_list.append(val_of_y)

In [10]:
"""
Creating a daataframe with the assets name, the best model used in each asset and the predicted returns 
for the n-12 periods
"""

final_results = pd.DataFrame()
final_results['Asset'] = asset_list
final_results['Model'] = best_model_list
final_results['Returns_predict'] = val_of_y_list

In [11]:
final_results

,Asset,Model,Returns_predict
0,HES US Equity,RandomForestRegressor,"[-2.809064600000006, 0.6152145699999998, -4.06..."
1,NEM US Equity,RandomForestRegressor,"[-1.7914076059999997, -0.9033056110000004, -4...."
2,INTC US Equity,LGBMRegressor,"[3.1969774527983232, 3.313437153895696, 3.1351..."
3,IFF US Equity,XGBRegressor,"[2.137643337249756, 4.116685390472412, 0.60484..."
4,MOS US Equity,XGBRegressor,"[-0.2642870545387268, -8.006417274475098, 7.39..."
5,EXC US Equity,AdaBoostRegressor,"[-0.9027720666666665, -0.8123149333333334, -0...."
6,KMB US Equity,RandomForestRegressor,"[0.4350848799999998, -0.16565399600000016, -0...."
7,JNJ US Equity,LGBMRegressor,"[-0.12326520410451022, -0.41003084431091946, 0..."
8,BAX US Equity,SGDRegressor,"[2.6847617651458355, 4.176697672749186, -4.863..."
